# Collecting data from ThermoML xml files

In [2]:
import os
from ThermoML import ThermoMLBuilder
import polars as pl
import numpy as np
from importlib import reload
import glob

thermoml_schema.py generator:

In [3]:
# pyxbgen -u ThermoML/ThermoML.xsd -m ThermoML/thermoml_schema

In [4]:
def RetrieveFiles():
    """Outputs a list of files path from reference DOIs (taken from https://trc.nist.gov/ThermoML/Browse)
    
    Parameters
    ----------
    ref: str
        file path for the file with the reference DOIs

    Outputs
    ----------
    files: list(str)
        list of paths for each reference file
    """


    dir = os.getcwd()
    files = [os.path.join(dir, file)
                for file in glob.glob('ThermoML/*/*.xml')]
    return files


## Raw data

Collecting data from all files

In [5]:
files = RetrieveFiles() 
files[0]

'/home/wildsonbbl/documents/code/themoml/ThermoML/10.1016/j.fluid.2019.06.015.xml'

In [9]:
data = ThermoMLBuilder.build_dataset(files,'data','raw')

files: 100%|██████████| 11923/11923 [3:21:53<00:00,  1.02s/it]   
